In [8]:
# ================================
# Algorithms and Utility Functions
# ================================

import numpy as np
from math import log, ceil, floor, log2, exp
from collections import defaultdict

def hoeffding_confidence(chosen_arm, sample_means, num_pulls):
    """
    Calculates Hoeffding-based lower bound confidence for the selected arm.
    """
    chosen_mean = sample_means.get(chosen_arm, 0)
    confidences = []
    for arm, mean in sample_means.items():
        if arm == chosen_arm:
            continue
        delta = chosen_mean - mean
        if delta <= 0:
            confidences.append(0.0)
        else:
            n = min(num_pulls.get(arm, 1), num_pulls.get(chosen_arm, 1))
            conf = 1 - exp(-0.5 * n * delta**2)
            confidences.append(conf)
    return min(confidences) if confidences else 1.0

def exponential_gap_elimination(arm_pulls, delta=0.05):
    """
    EGE algorithm for fixed-confidence best arm identification.
    """
    S = list(arm_pulls.keys())
    r = 1
    total_cost = 0
    total_pulls = 0
    sample_means = {}
    num_pulls = {}
    stopping_early = False

    while len(S) > 1:
        ε_r = 2 ** (-r / 4)
        δ_r = delta / (50 * r ** 3)
        t_r = ceil((2 / (ε_r ** 2)) * log(2 / δ_r))

        max_available = min([len(arm_pulls[arm]) - num_pulls.get(arm, 0) for arm in S])
        if max_available == 0:
            stopping_early = True
            break
        t_r = min(t_r, max_available)

        round_means = {}
        valid_arms = []

        for arm in S:
            start = num_pulls.get(arm, 0)
            available = len(arm_pulls[arm]) - start
            arm_tr = min(t_r, available)
            if arm_tr <= 0:
                continue

            pulls = arm_pulls[arm][start:start + arm_tr]
            rewards = [r for r, _ in pulls]
            costs = [c for _, c in pulls]

            if not rewards:
                continue

            avg = np.mean(rewards)
            sample_means[arm] = avg
            round_means[arm] = avg
            num_pulls[arm] = start + len(rewards)
            total_cost += sum(costs)
            total_pulls += len(rewards)
            valid_arms.append(arm)

        if len(valid_arms) == 0:
            stopping_early = True
            break

        best_arm = max(round_means, key=round_means.get)
        p_hat_star = round_means[best_arm]
        S = [arm for arm in valid_arms if round_means[arm] >= p_hat_star - ε_r]
        r += 1

    chosen_arm = S[0] if S else None
    confidence = hoeffding_confidence(chosen_arm, sample_means, num_pulls)
    return chosen_arm, total_pulls, total_cost, confidence, stopping_early

def sequential_halving(arm_pulls, total_budget):
    """
    SH algorithm for fixed-budget best arm identification.
    """
    S = list(arm_pulls.keys())
    R = ceil(log2(len(S)))
    budget_used = 0
    total_pulls = 0
    sample_means = {}
    num_pulls = {}
    stopping_early = False

    for r in range(R):
        if len(S) <= 1:
            break

        avg_costs = {arm: np.mean([c for _, c in arm_pulls[arm]]) for arm in S}
        mean_cost_per_arm = sum(avg_costs.values()) / len(S)
        max_total_pulls = floor((total_budget - budget_used) / mean_cost_per_arm)
        t_r = max(floor(max_total_pulls / len(S)), 1)

        max_available = min([len(arm_pulls[arm]) - num_pulls.get(arm, 0) for arm in S])
        if max_available == 0:
            stopping_early = True
            break
        t_r = min(t_r, max_available)

        round_means = {}
        valid_arms = []

        for arm in S:
            start = num_pulls.get(arm, 0)
            available = len(arm_pulls[arm]) - start
            arm_tr = min(t_r, available)
            if arm_tr <= 0:
                continue

            pulls = arm_pulls[arm][start:start + arm_tr]
            rewards = [r for r, _ in pulls]
            costs = [c for _, c in pulls]
            cost_sum = sum(costs)

            if not rewards or (budget_used + cost_sum > total_budget):
                continue

            sample_means[arm] = np.mean(rewards)
            round_means[arm] = sample_means[arm]
            num_pulls[arm] = start + len(rewards)
            budget_used += cost_sum
            total_pulls += len(rewards)
            valid_arms.append(arm)

        if len(valid_arms) == 0:
            stopping_early = True
            break

        sorted_arms = sorted(round_means.items(), key=lambda x: x[1], reverse=True)
        S = [arm for arm, _ in sorted_arms[:ceil(len(valid_arms) / 2)]]

        if budget_used >= total_budget:
            break

    chosen_arm = S[0] if S else None
    confidence = hoeffding_confidence(chosen_arm, sample_means, num_pulls)
    return chosen_arm, total_pulls, budget_used, confidence, stopping_early


In [10]:
# ================================
# Experiment Runner and Generator
# ================================

import random
from tqdm.auto import tqdm

# Adjustable Parameters
NUM_CASES = 5000
NUM_ARMS = 6
NUM_PULLS_PER_ARM = 1500
COST_RANGE = (0.05, 0.2)
REWARD_GAP_MIN = 0.1
DELTA = 0.05
BUDGET = 1000

def generate_case(normal_gap=REWARD_GAP_MIN):
    """
    Generate one synthetic case with 6 arms having normal rewards and random costs.
    """
    best_mean = round(random.uniform(0.6, 0.9), 3)
    other_means = [best_mean - random.uniform(normal_gap, normal_gap + 0.1) for _ in range(NUM_ARMS - 1)]
    all_means = [best_mean] + other_means
    random.shuffle(all_means)

    std_devs = [round(random.uniform(0.05, 0.15), 3) for _ in range(NUM_ARMS)]
    pulls = defaultdict(list)
    for arm in range(NUM_ARMS):
        rewards = np.random.normal(loc=all_means[arm], scale=std_devs[arm], size=NUM_PULLS_PER_ARM)
        costs = np.random.uniform(*COST_RANGE, size=NUM_PULLS_PER_ARM)
        pulls[arm] = list(zip(rewards, costs))

    return pulls, all_means, std_devs

def run_experiments(num_cases=5000):
    ege_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    sh_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    all_means, all_stds = [], []

    for _ in tqdm(range(num_cases)):
        arm_pulls, means, stds = generate_case()
        all_means.extend(means)
        all_stds.extend(stds)
        best_arm = int(np.argmax(means))

        # Run EGE
        ege_arm, ep, ec, ecf, early_e = exponential_gap_elimination(arm_pulls, delta=DELTA)
        ege_results['pulls'].append(ep)
        ege_results['cost'].append(ec)
        ege_results['confidence'].append(ecf)
        ege_results['correct'] += (ege_arm == best_arm)
        ege_results['early'] += early_e

        # Run SH
        sh_arm, sp, sc, scf, early_s = sequential_halving(arm_pulls, total_budget=BUDGET)
        sh_results['pulls'].append(sp)
        sh_results['cost'].append(sc)
        sh_results['confidence'].append(scf)
        sh_results['correct'] += (sh_arm == best_arm)
        sh_results['early'] += early_s

    return ege_results, sh_results, all_means, all_stds

def print_summary(results, num_cases):
    def summarize(res):
        return {
            'Avg Pulls': np.mean(res['pulls']),
            'Avg Cost': np.mean(res['cost']),
            'Avg Confidence': np.mean(res['confidence']),
            'Accuracy (%)': 100 * res['correct'] / num_cases,
            'Stopping Early (%)': 100 * res['early'] / num_cases
        }

    ege_res, sh_res, means, stds = results
    print("\n=== Exponential Gap Elimination ===")
    print(summarize(ege_res))

    print("\n=== Sequential Halving ===")
    print(summarize(sh_res))

    print("\n=== Distribution Summary ===")
    print(f"Mean of reward means: {np.mean(means):.4f}")
    print(f"Mean of std devs: {np.mean(stds):.4f}")# ========================================
# Experiment Runner and Readable Formatter
# ========================================

import random
import pandas as pd
from tqdm.auto import tqdm

# Adjustable Parameters
NUM_CASES = 5000
NUM_ARMS = 6
NUM_PULLS_PER_ARM = 1500
COST_RANGE = (0.05, 0.2)
REWARD_GAP_MIN = 0.1
DELTA = 0.05
BUDGET = 1000

def generate_case(normal_gap=REWARD_GAP_MIN):
    """
    Generate one synthetic case with 6 arms having normal rewards and random costs.
    Ensures only one best arm with reward gap ≥ 0.1.
    """
    best_mean = round(random.uniform(0.6, 0.9), 3)
    other_means = [best_mean - random.uniform(normal_gap, normal_gap + 0.1) for _ in range(NUM_ARMS - 1)]
    all_means = [best_mean] + other_means
    random.shuffle(all_means)

    std_devs = [round(random.uniform(0.05, 0.15), 3) for _ in range(NUM_ARMS)]
    pulls = defaultdict(list)
    for arm in range(NUM_ARMS):
        rewards = np.random.normal(loc=all_means[arm], scale=std_devs[arm], size=NUM_PULLS_PER_ARM)
        costs = np.random.uniform(*COST_RANGE, size=NUM_PULLS_PER_ARM)
        pulls[arm] = list(zip(rewards, costs))

    return pulls, all_means, std_devs

def run_experiments(num_cases=5000):
    ege_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    sh_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    all_means, all_stds = [], []

    for _ in tqdm(range(num_cases)):
        arm_pulls, means, stds = generate_case()
        all_means.extend(means)
        all_stds.extend(stds)
        best_arm = int(np.argmax(means))

        # Run EGE
        ege_arm, ep, ec, ecf, early_e = exponential_gap_elimination(arm_pulls, delta=DELTA)
        ege_results['pulls'].append(ep)
        ege_results['cost'].append(ec)
        ege_results['confidence'].append(ecf)
        ege_results['correct'] += (ege_arm == best_arm)
        ege_results['early'] += early_e

        # Run SH
        sh_arm, sp, sc, scf, early_s = sequential_halving(arm_pulls, total_budget=BUDGET)
        sh_results['pulls'].append(sp)
        sh_results['cost'].append(sc)
        sh_results['confidence'].append(scf)
        sh_results['correct'] += (sh_arm == best_arm)
        sh_results['early'] += early_s

    return ege_results, sh_results, all_means, all_stds

def format_summary(results, num_cases):
    """
    Convert summary stats into a neat DataFrame.
    """
    def summarize(res):
        return {
            "Average Pulls": np.mean(res['pulls']),
            "Average Cost": np.mean(res['cost']),
            "Average Confidence": np.mean(res['confidence']),
            "Accuracy (%)": 100 * res['correct'] / num_cases,
            "Stopping Early (%)": 100 * res['early'] / num_cases
        }

    ege_stats = summarize(results[0])
    sh_stats = summarize(results[1])
    df_summary = pd.DataFrame([ege_stats, sh_stats], index=["EGE", "SH"])
    df_summary = df_summary.round(2)

    dist_summary = {
        "Avg of Arm Means": np.mean(results[2]),
        "Avg of Arm StdDevs": np.mean(results[3])
    }
    dist_df = pd.DataFrame([dist_summary], index=["Distribution Stats"]).round(3)

    return df_summary, dist_df

# Run Experiments
results = run_experiments(NUM_CASES)

# Display formatted summary
summary_df, dist_df = format_summary(results, NUM_CASES)

print("\n=== Algorithm Performance Summary ===")
display(summary_df)

print("\n=== Reward Distribution Summary ===")
display(dist_df)


# Run Experiments
results = run_experiments(NUM_CASES)
print_summary(results, NUM_CASES)


  0%|          | 0/5000 [00:00<?, ?it/s]


=== Algorithm Performance Summary ===


,Average Pulls,Average Cost,Average Confidence,Accuracy (%),Stopping Early (%)
EGE,9000.00,1124.95,0.16,16.38,100.00
SH,7677.76,959.35,0.80,81.22,52.14



=== Reward Distribution Summary ===


,Avg of Arm Means,Avg of Arm StdDevs
Distribution Stats,0.625,0.1


  0%|          | 0/5000 [00:00<?, ?it/s]


=== Exponential Gap Elimination ===
{'Avg Pulls': np.float64(9000.0), 'Avg Cost': np.float64(1125.0505991343493), 'Avg Confidence': np.float64(0.16430129741144786), 'Accuracy (%)': 16.44, 'Stopping Early (%)': 100.0}

=== Sequential Halving ===
{'Avg Pulls': np.float64(7669.905), 'Avg Cost': np.float64(958.4392966627129), 'Avg Confidence': np.float64(0.8007523006429996), 'Accuracy (%)': 82.36, 'Stopping Early (%)': 51.8}

=== Distribution Summary ===
Mean of reward means: 0.6241
Mean of std devs: 0.0998


In [11]:
# ================================
# Experiment Runner and Generator
# ================================

import random
from tqdm.auto import tqdm

# Adjustable Parameters
NUM_CASES = 5000
NUM_ARMS = 6
NUM_PULLS_PER_ARM = 1500
COST_RANGE = (0.05, 0.2)
REWARD_GAP_MIN = 0.1
DELTA = 0.05
BUDGET = 1000

def generate_case(normal_gap=REWARD_GAP_MIN):
    """
    Generate one synthetic case with 6 arms having normal rewards and random costs.
    """
    best_mean = round(random.uniform(0.6, 0.9), 3)
    other_means = [best_mean - random.uniform(normal_gap, normal_gap + 0.1) for _ in range(NUM_ARMS - 1)]
    all_means = [best_mean] + other_means
    random.shuffle(all_means)

    std_devs = [round(random.uniform(0.05, 0.15), 3) for _ in range(NUM_ARMS)]
    pulls = defaultdict(list)
    for arm in range(NUM_ARMS):
        rewards = np.random.normal(loc=all_means[arm], scale=std_devs[arm], size=NUM_PULLS_PER_ARM)
        costs = np.random.uniform(*COST_RANGE, size=NUM_PULLS_PER_ARM)
        pulls[arm] = list(zip(rewards, costs))

    return pulls, all_means, std_devs

def run_experiments(num_cases=5000):
    ege_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    sh_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    all_means, all_stds = [], []

    for _ in tqdm(range(num_cases)):
        arm_pulls, means, stds = generate_case()
        all_means.extend(means)
        all_stds.extend(stds)
        best_arm = int(np.argmax(means))

        # Run EGE
        ege_arm, ep, ec, ecf, early_e = exponential_gap_elimination(arm_pulls, delta=DELTA)
        ege_results['pulls'].append(ep)
        ege_results['cost'].append(ec)
        ege_results['confidence'].append(ecf)
        ege_results['correct'] += (ege_arm == best_arm)
        ege_results['early'] += early_e

        # Run SH
        sh_arm, sp, sc, scf, early_s = sequential_halving(arm_pulls, total_budget=BUDGET)
        sh_results['pulls'].append(sp)
        sh_results['cost'].append(sc)
        sh_results['confidence'].append(scf)
        sh_results['correct'] += (sh_arm == best_arm)
        sh_results['early'] += early_s

    return ege_results, sh_results, all_means, all_stds

def print_summary(results, num_cases):
    def summarize(res):
        return {
            'Avg Pulls': np.mean(res['pulls']),
            'Avg Cost': np.mean(res['cost']),
            'Avg Confidence': np.mean(res['confidence']),
            'Accuracy (%)': 100 * res['correct'] / num_cases,
            'Stopping Early (%)': 100 * res['early'] / num_cases
        }

    ege_res, sh_res, means, stds = results
    print("\n=== Exponential Gap Elimination ===")
    print(summarize(ege_res))

    print("\n=== Sequential Halving ===")
    print(summarize(sh_res))

    print("\n=== Distribution Summary ===")
    print(f"Mean of reward means: {np.mean(means):.4f}")
    print(f"Mean of std devs: {np.mean(stds):.4f}")# ========================================
# Experiment Runner and Readable Formatter
# ========================================

import random
import pandas as pd
from tqdm.auto import tqdm

# Adjustable Parameters
NUM_CASES = 5000
NUM_ARMS = 6
NUM_PULLS_PER_ARM = 1500
COST_RANGE = (0.05, 0.2)
REWARD_GAP_MIN = 0.2
DELTA = 0.05
BUDGET = 1000

def generate_case(normal_gap=REWARD_GAP_MIN):
    """
    Generate one synthetic case with 6 arms having normal rewards and random costs.
    Ensures only one best arm with reward gap ≥ 0.1.
    """
    best_mean = round(random.uniform(0.6, 0.9), 3)
    other_means = [best_mean - random.uniform(normal_gap, normal_gap + 0.1) for _ in range(NUM_ARMS - 1)]
    all_means = [best_mean] + other_means
    random.shuffle(all_means)

    std_devs = [round(random.uniform(0.05, 0.15), 3) for _ in range(NUM_ARMS)]
    pulls = defaultdict(list)
    for arm in range(NUM_ARMS):
        rewards = np.random.normal(loc=all_means[arm], scale=std_devs[arm], size=NUM_PULLS_PER_ARM)
        costs = np.random.uniform(*COST_RANGE, size=NUM_PULLS_PER_ARM)
        pulls[arm] = list(zip(rewards, costs))

    return pulls, all_means, std_devs

def run_experiments(num_cases=5000):
    ege_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    sh_results = {'pulls': [], 'cost': [], 'confidence': [], 'correct': 0, 'early': 0}
    all_means, all_stds = [], []

    for _ in tqdm(range(num_cases)):
        arm_pulls, means, stds = generate_case()
        all_means.extend(means)
        all_stds.extend(stds)
        best_arm = int(np.argmax(means))

        # Run EGE
        ege_arm, ep, ec, ecf, early_e = exponential_gap_elimination(arm_pulls, delta=DELTA)
        ege_results['pulls'].append(ep)
        ege_results['cost'].append(ec)
        ege_results['confidence'].append(ecf)
        ege_results['correct'] += (ege_arm == best_arm)
        ege_results['early'] += early_e

        # Run SH
        sh_arm, sp, sc, scf, early_s = sequential_halving(arm_pulls, total_budget=BUDGET)
        sh_results['pulls'].append(sp)
        sh_results['cost'].append(sc)
        sh_results['confidence'].append(scf)
        sh_results['correct'] += (sh_arm == best_arm)
        sh_results['early'] += early_s

    return ege_results, sh_results, all_means, all_stds

def format_summary(results, num_cases):
    """
    Convert summary stats into a neat DataFrame.
    """
    def summarize(res):
        return {
            "Average Pulls": np.mean(res['pulls']),
            "Average Cost": np.mean(res['cost']),
            "Average Confidence": np.mean(res['confidence']),
            "Accuracy (%)": 100 * res['correct'] / num_cases,
            "Stopping Early (%)": 100 * res['early'] / num_cases
        }

    ege_stats = summarize(results[0])
    sh_stats = summarize(results[1])
    df_summary = pd.DataFrame([ege_stats, sh_stats], index=["EGE", "SH"])
    df_summary = df_summary.round(2)

    dist_summary = {
        "Avg of Arm Means": np.mean(results[2]),
        "Avg of Arm StdDevs": np.mean(results[3])
    }
    dist_df = pd.DataFrame([dist_summary], index=["Distribution Stats"]).round(3)

    return df_summary, dist_df

# Run Experiments
results = run_experiments(NUM_CASES)

# Display formatted summary
summary_df, dist_df = format_summary(results, NUM_CASES)

print("\n=== Algorithm Performance Summary ===")
display(summary_df)

print("\n=== Reward Distribution Summary ===")
display(dist_df)


# Run Experiments
results = run_experiments(NUM_CASES)
print_summary(results, NUM_CASES)


  0%|          | 0/5000 [00:00<?, ?it/s]


=== Algorithm Performance Summary ===


,Average Pulls,Average Cost,Average Confidence,Accuracy (%),Stopping Early (%)
EGE,8417.15,1052.16,0.77,76.58,42.02
SH,7681.46,959.85,0.87,86.78,51.10



=== Reward Distribution Summary ===


,Avg of Arm Means,Avg of Arm StdDevs
Distribution Stats,0.542,0.1


  0%|          | 0/5000 [00:00<?, ?it/s]


=== Exponential Gap Elimination ===
{'Avg Pulls': np.float64(8421.3154), 'Avg Cost': np.float64(1052.7343376651488), 'Avg Confidence': np.float64(0.7689999999991606), 'Accuracy (%)': 76.9, 'Stopping Early (%)': 42.22}

=== Sequential Halving ===
{'Avg Pulls': np.float64(7681.8272), 'Avg Cost': np.float64(959.9513630958036), 'Avg Confidence': np.float64(0.8696016767466553), 'Accuracy (%)': 86.88, 'Stopping Early (%)': 50.44}

=== Distribution Summary ===
Mean of reward means: 0.5422
Mean of std devs: 0.1000
